In [16]:
import folium
import pandas as pd

In [17]:
file_path = '/home/lolomarka/Documents/web-analysis/lab4/metro.tsv'
data = pd.read_csv(file_path, sep='\t')
data

,Название,Широта,Долгота
0,Аэропорт,55.799810,37.534120
1,Академическая,55.688080,37.575010
2,Алексеевская,55.807370,37.638440
3,Александровский сад,55.752190,37.608360
4,Алтуфьево,55.895040,37.586050
...,...,...,...
160,Воробьёвы горы,55.710454,37.558601
161,Выхино,55.715000,37.818020
162,Ясенево,55.605350,37.534940
163,Юго-западная,55.664640,37.484210


In [18]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from pandas import DataFrame

def find_metros(url : str) -> list:
    response = requests.get(url, verify=False)
    lst = list()
    if response.status_code == 200:
        soup = BeautifulSoup(response.content.decode('utf-8', 'ignore'), 'html.parser')
        for div in soup.find_all('div', class_='features-item'):
            title = div.find('span', class_='features-title')
            if title:
                text = title.text
                if text.endswith(','):
                    text = text[:-1]
                lst.append(text.strip())
    return lst

def get_districts() -> dict:
    url = 'https://domokucha.info/moscow_metro_district.html'
    response = requests.get(url, verify=False)
    districts = {}
    if response.status_code == 200:
        soup = BeautifulSoup(response.content.decode('utf-8', 'ignore'), 'html.parser')
        
        for item in soup.find_all('div', class_='features-item'):
            href = item.find('a', href=True)['href']
            if href.startswith('./moscow_metro'):
                district = item.find('span', class_='features-title').text
                for station in find_metros(urljoin(url, href)):
                    districts[station.lower()] = district 
              
    return districts
# get_districts()

/home/lolomarka/Documents/web-analysis/.venv/lib64/python3.13/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domokucha.info'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/lolomarka/Documents/web-analysis/.venv/lib64/python3.13/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domokucha.info'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/lolomarka/Documents/web-analysis/.venv/lib64/python3.13/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domokucha.info'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usa

{'щёлковская': 'Восточный административный округ',
 'первомайская': 'Восточный административный округ',
 'измайловская': 'Восточный административный округ',
 'партизанская': 'Восточный административный округ',
 'семеновская': 'Восточный административный округ',
 'электрозаводская': 'Восточный административный округ',
 'новокосино': 'Восточный административный округ',
 'новогиреево': 'Восточный административный округ',
 'перово': 'Восточный административный округ',
 'шоссе энтузиастов': 'Восточный административный округ',
 'бульвар рокоссовского': 'Восточный административный округ',
 'черкизовская': 'Восточный административный округ',
 'преображенская площадь': 'Восточный административный округ',
 'сокольники': 'Восточный административный округ',
 'крылатское': 'Западный административный округ',
 'молодежная': 'Западный административный округ',
 'кунцевская': 'Западный административный округ',
 'славянский бульвар': 'Западный административный округ',
 'парк победы': 'Западный администра

In [20]:
def generate_map_and_show() -> None:
    map = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

    district_colors = {
        'Центральный административный округ': "blue",
        'Северный административный округ': "green",
        'Юго-Западный административный округ': "orange",
        'Северо-Восточный административный округ': "purple",
        'Южный административный округ': "red",
        'Восточный административный округ': "beige",
        'Западный административный округ': "lightgreen",
        'Северо-Западный административный округ': "darkblue",
        'Юго-Восточный административный округ': "pink"
    }
    districts_station = get_districts()
    for i, row in data.iterrows():
        station = row['Название']

        district = districts_station.get(station.lower(), 'NOT FOUND')
        color = district_colors.get(district, 'lightgray')
        lat = row['Широта']
        long = row['Долгота']
        folium.Marker(location=[lat, long], popup=f'{station} ({lat}, {long}) - {district}', icon=folium.Icon(color=color)).add_to(map)

    map_file = '/home/lolomarka/Documents/web-analysis/lab4/map.html'
    map.save(map_file)
    display(map)

generate_map_and_show()

/home/lolomarka/Documents/web-analysis/.venv/lib64/python3.13/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domokucha.info'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/lolomarka/Documents/web-analysis/.venv/lib64/python3.13/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domokucha.info'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/lolomarka/Documents/web-analysis/.venv/lib64/python3.13/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domokucha.info'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usa